# General

In [1]:
from package import process_dataset
from package import encoder
from package import models

import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
description = pd.read_csv("data/dicionario.csv", encoding='latin-1')
train = pd.read_csv("data/dados_treino_hackaton.csv", encoding='latin-1', index_col=False)
test = pd.read_csv("data/dados_teste_x_hackaton.csv", encoding='latin-1', index_col=False)

del train['Unnamed: 0']
del test['Unnamed: 0']

In [2]:
train = encoder.encode_DataFrame(train)
train = process_dataset.processColumns(train)

NameError: name 'encoder' is not defined

In [250]:
train_cols, target = process_dataset.generate_labels(train)

In [251]:
X = process_dataset.scaleData(train[train_cols], useSaved=False)
y = train[target]

### Variando size

In [270]:
#podemos variar o test_size de 0.1 até 0.2 ver como os modelos treinam melhor
size = 0.12
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)

# Decision tree

In [202]:
from sklearn.tree import DecisionTreeRegressor

In [203]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [204]:
pred = tree.predict(X_test)

In [205]:
convPred = models.convertPredicted(pred, threshold=0.3)

In [206]:
models.accuracy_score(y_test, convPred)

0.6666666666666666

In [218]:
lm = 0
f = open('testesDT.txt', 'a')
for md in [None, 1, 2, 3, 4, 5]:
    for mss in np.arange(0.1, 1, 0.05):
        for split in ["best", "random"]:
            for mf in [None,0.1,5,10,12,32,"sqrt","auto","log2"]:
                for mid in np.arange(0, 5, 0.5):
                    for ccp in np.arange(0,5,0.5):
                        for msf in np.arange(0.1,5,0.5):
                            tree = DecisionTreeRegressor(
                                max_depth= md, 
                                min_samples_split= mss, 
                                splitter= split, 
                                max_features= mf, 
                                min_impurity_decrease = mid,
                                ccp_alpha = ccp,
                                min_samples_leaf = msf
                            )
                            tree.fit(X_train, y_train)
                            pred = tree.predict(X_test)
                            convPred = models.convertPredicted(pred, threshold=0.5)
                            r = models.accuracy_score(y_test, convPred)
                            if r > lm:
                                lm = r
                                f.write(f'Higher = {lm}\n    test_size = {size}, max_depth = {md}, min_samples_split = {mss}, splitter = {split}, max_features = {mf}, min_impurity_decrease = {mid}, ccp_alpha = {ccp}, min_samples_leaf = {msf}\n')

                                print("HIGHER ", lm)
                                print(md, mss, split, mf, mid, ccp, msf, size) 
                                print("")
f.close()

HIGHER  0.7073170731707317
None 0.1 best None 0.0 0.0 0.1

HIGHER  0.7317073170731707
None 0.15000000000000002 random 10 0.0 0.0 0.1

HIGHER  0.7398373983739838
2 0.5000000000000001 random None 0.0 0.0 0.1



KeyboardInterrupt: 

In [223]:
f.close()

# AdaBosst

In [241]:
from sklearn.ensemble import AdaBoostRegressor

In [280]:
lm = 0
f = open('testesAB.txt', 'a')
for be in [None]:
    for ne in np.arange(300, 150, -1):
        for lr in np.arange(100, 1, -1):
            for l in ["linear", "square", "exponential"]:
                ada = AdaBoostRegressor(
                    base_estimator = be, 
                    n_estimators = ne, 
                    learning_rate = lr, 
                    loss = l, 
                    random_state = rs
                )
                ada.fit(X_train, y_train)
                pred = ada.predict(X_test)
                convPred = models.convertPredicted(pred, threshold=0.5)
                r = models.accuracy_score(y_test, convPred)
                if r > lm and r > 0.75:
                    lm = r
                    f.write(f'Higher = {lm}\n    test_size = {size}, base_estimator = {be}, n_estimators = {ne}, learning_rate = {lr}, loss = {l}, random_state = {rs}\n')

                    print("HIGHER ", lm)
                    print(be, ne, lr, l, rs, size) 
                    print("")
f.close()

HIGHER  0.7702702702702703
None 300 2 square 94 0.12



KeyboardInterrupt: 

In [281]:
f.close()

# LightGBM

In [1]:
from lightgbm import LGBMClassifier

In [307]:
lm = 0
f = open('testesRFR.txt', 'a')

for ne in np.arange(1, 100, 1):
    for crit in ["squared_error", "absolute_error", "poisson"]:
        for md in np.arange(1, 10, 2):
            for mss in np.arange(0.1, 5, 0.5):
                for mwfl in np.arange(0, 0.5, 0.1):
                    for mf in ["sqrt", "log2", None]:
                        for mln in [2, 5, 12, 32, 43, None]:
                            for mid in np.arange(0, 5, 0.5):
                                for btp in [True, False]:
                                    for ccpa in np.arange(0, 5, 0.5):
                                        for ms in [None, 0.2, 0.5, 0.9]:
                                            if btp == False: ms = None
                                            lg = LGBMClassifier(
                                                n_estimators = ne, 
                                                criterion = crit, 
                                                max_depth = md, 
                                                min_samples_split = mss, 
                                                min_weight_fraction_leaf = mwfl,
                                                max_features = mf,
                                                max_leaf_nodes = mln,
                                                min_impurity_decrease = mid,
                                                bootstrap = btp,
                                                n_jobs = None,
                                                ccp_alpha = ccpa,
                                                max_samples = ms
                                            )
                                            lg.fit(X_train, y_train)
                                            pred = lg.predict(X_test)
                                            convPred = models.convertPredicted(pred, threshold=0.5)
                                            r = models.accuracy_score(y_test, convPred)
                                            if r > lm and r > 0.75:
                                                lm = r
                                                f.write(f'Higher = {lm}\n    test_size = {size}, n_estimators = {ne}, max_depth = {md}, min_samples_split = {mss}, min_weight_fraction_leaf = {mwfl}, max_features = {mf}, max_leaf_nodes = {mln}, min_impurity_decrease = {mid}, bootstrap = {btp}, oob_score = {os}, random_state = {rs}, verbose = {vb}, ccp_alpha = {ccpa}, max_samples = {ms}\n')

                                                print("HIGHER ", lm)
                                                print(be, ne, lr, l, rs, size) 
                                                print("")
f.close()

KeyboardInterrupt: 

In [291]:
f.close()